In [4]:
import pandas as pd
from lxml import etree as et
import os
from datetime import date as dt

def parse_all_xml():
    def extracao_nf_versao_1_0():
        serie_nfe = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[5].text
        num_nfe = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[6].text
        dt_emissao = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[7].text
        tipo_comprador = root.getchildren()[0].getchildren()[0].getchildren()[2].getchildren()[0].tag
        tipo_comprador = tipo_comprador.replace('{http://www.portalfiscal.inf.br/nfe}', '')
        uf_comprador = root.getchildren()[0].getchildren()[0].getchildren()[2].getchildren()[2].findall('{http://www.portalfiscal.inf.br/nfe}UF')[0].text
        v_frete = float(root.getchildren()[0].getchildren()[0].getchildren()[5].getchildren()[0].getchildren()[5].text)

        container =  root.getchildren()[0].getchildren()[0].getchildren()
        produtos = []
        for element in container:
            if element.tag == '{http://www.portalfiscal.inf.br/nfe}det':
                produtos.append(element)
        lst_sku_prod  = []
        lst_nome_prod = []
        lst_qtd_prod  = []
        lst_vl_unit_prod = []
        lst_sub_total_prod = []
        for prod in produtos:
            sku_prod = prod.getchildren()[0].getchildren()[0].text
            lst_sku_prod.append(sku_prod)
            nome_prod = prod.getchildren()[0].getchildren()[2].text
            lst_nome_prod.append(nome_prod)
            qtd_prod = float(prod.getchildren()[0].getchildren()[5].text)
            lst_qtd_prod.append(qtd_prod)
            vl_unit_prod = float(prod.getchildren()[0].getchildren()[6].text)
            lst_vl_unit_prod.append(vl_unit_prod)
            sub_total_prod = float(prod.getchildren()[0].getchildren()[7].text)
            lst_sub_total_prod.append(sub_total_prod)

        lista_de_colunas_df = ['xml_version', 'dt_emissao', 'num_nfe', 'tipo_comprador', 'uf', 'sku_prod', 'nome_prod', 'qtd', 'vl_unit',   'sub_total']
        df = pd.DataFrame(index=range(len(lst_sku_prod) ), columns=lista_de_colunas_df)
        df['xml_version'] = versao_xml
        df['dt_emissao'] = dt_emissao
        df['num_nfe'] = num_nfe
        df['tipo_comprador'] = tipo_comprador
        df['uf'] = uf_comprador
        df['sku_prod'] = lst_sku_prod
        df['nome_prod'] = lst_nome_prod
        df['qtd'] = lst_qtd_prod
        df['vl_unit'] = lst_vl_unit_prod
        df['sub_total'] = df['qtd'] * df['vl_unit']
        #Ajustes de Objects
        #df['dt_emissao'] = df['dt_emissao'].str.split('T').str[0]
        #df['dt_emissao'] = pd.to_datetime(df['dt_emissao'], format="%Y-%m-%d")
        
        return df


    
    def extracao_nf_versao_2_0():
        serie_nfe = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[4].text
        num_nfe = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[6].text
        dt_emissao = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[7].text
        tipo_comprador = root.getchildren()[0].getchildren()[0].getchildren()[2].getchildren()[0].tag
        tipo_comprador = tipo_comprador.replace('{http://www.portalfiscal.inf.br/nfe}', '')
        uf_comprador = root.getchildren()[0].getchildren()[0].getchildren()[2].getchildren()[2].findall('{http://www.portalfiscal.inf.br/nfe}UF')[0].text
        v_frete = float(root.getchildren()[0].getchildren()[0].findall('{http://www.portalfiscal.inf.br/nfe}total')[0].getchildren()[0].findall('{http://www.portalfiscal.inf.br/nfe}vFrete')[0].text)
        # Extração de Produtos do XML
        container =  root.getchildren()[0].getchildren()[0].getchildren()
        produtos = []
        for element in container:
            if element.tag == '{http://www.portalfiscal.inf.br/nfe}det':
                produtos.append(element)
        lst_sku_prod  = []
        lst_nome_prod = []
        lst_qtd_prod  = []
        lst_vl_unit_prod = []
        lst_sub_total_prod = []
        for prod in produtos:
            sku_prod = prod.getchildren()[0].getchildren()[0].text
            lst_sku_prod.append(sku_prod)
            nome_prod = prod.getchildren()[0].getchildren()[2].text
            lst_nome_prod.append(nome_prod)
            qtd_prod = float(prod.getchildren()[0].getchildren()[6].text)
            lst_qtd_prod.append(qtd_prod)

            vl_unit_prod = prod.getchildren()[0].getchildren()[7].text
            lst_vl_unit_prod.append(vl_unit_prod)
            sub_total_prod = prod.getchildren()[0].getchildren()[8].text
            lst_sub_total_prod.append(sub_total_prod)

            # Montagem DF
        lista_de_colunas_df = ['xml_version', 'dt_emissao', 'num_nfe', 'tipo_comprador', 'uf', 'sku_prod', 'nome_prod', 'qtd', 'vl_unit', 'sub_total']
        df = pd.DataFrame(index=range(len(lst_sku_prod) ), columns=lista_de_colunas_df)
        df['xml_version'] = versao_xml
        df['dt_emissao'] = dt_emissao
        df['num_nfe'] = num_nfe
        df['tipo_comprador'] = tipo_comprador
        df['uf'] = uf_comprador
        df['sku_prod'] = lst_sku_prod
        df['nome_prod'] = lst_nome_prod
        df['qtd'] = lst_qtd_prod
        df['vl_unit'] = lst_vl_unit_prod
        df['sub_total'] = lst_sub_total_prod
        #Ajustes de Objects
        #df['dt_emissao'] = df['dt_emissao'].str.split('T').str[0]
        #df['dt_emissao'] = pd.to_datetime(df['dt_emissao'], format="%Y-%m-%d")
        ##df[['vl_unit', 'sub_total']] = df[['vl_unit', 'sub_total']].replace({'\.':'', ',':''}, regex=True).astype(float) / 100
        #df['qtd'] = df['qtd'].str.split('\.').str[0]
        #df['qtd'] = df['qtd'].str.split(',').str[0].astype(int)
                
        return df


    def extracao_nf_versao_3_0():
        serie_nfe = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[4].text
        num_nfe = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[6].text
        dt_emissao = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[7].text
        tipo_comprador = root.getchildren()[0].getchildren()[0].getchildren()[2].getchildren()[0].tag
        tipo_comprador = tipo_comprador.replace('{http://www.portalfiscal.inf.br/nfe}', '')
        uf_comprador = root.getchildren()[0].getchildren()[0].getchildren()[2].getchildren()[2].findall('{http://www.portalfiscal.inf.br/nfe}UF')[0].text
        v_frete = float(root.getchildren()[0].getchildren()[0].findall('{http://www.portalfiscal.inf.br/nfe}total')[0].getchildren()[0].findall('{http://www.portalfiscal.inf.br/nfe}vFrete')[0].text)
        # Extração de Produtos do XML
        container =  root.getchildren()[0].getchildren()[0].getchildren()
        produtos = []
        for element in container:
            if element.tag == '{http://www.portalfiscal.inf.br/nfe}det':
                produtos.append(element)
        lst_sku_prod  = []
        lst_nome_prod = []
        lst_qtd_prod  = []
        lst_vl_unit_prod = []
        lst_sub_total_prod = []
        for prod in produtos:
            sku_prod = prod.getchildren()[0].getchildren()[0].text
            lst_sku_prod.append(sku_prod)
            nome_prod = prod.getchildren()[0].getchildren()[2].text
            lst_nome_prod.append(nome_prod)
            qtd_prod = prod.getchildren()[0].getchildren()[6].text
            lst_qtd_prod.append(qtd_prod)

            vl_unit_prod = prod.getchildren()[0].getchildren()[7].text
            lst_vl_unit_prod.append(vl_unit_prod)
            sub_total_prod = prod.getchildren()[0].getchildren()[8].text
            lst_sub_total_prod.append(sub_total_prod)

            # Montagem DF
        lista_de_colunas_df = ['xml_version', 'dt_emissao', 'num_nfe', 'tipo_comprador', 'uf', 'sku_prod', 'nome_prod', 'qtd', 'vl_unit', 'sub_total']
        df = pd.DataFrame(index=range(len(lst_sku_prod) ), columns=lista_de_colunas_df)
        df['xml_version'] = versao_xml
        df['dt_emissao'] = dt_emissao
        df['num_nfe'] = num_nfe
        df['tipo_comprador'] = tipo_comprador
        df['uf'] = uf_comprador
        df['sku_prod'] = lst_sku_prod
        df['nome_prod'] = lst_nome_prod
        df['qtd'] = lst_qtd_prod
        df['vl_unit'] = lst_vl_unit_prod
        df['sub_total'] = lst_sub_total_prod
        #Ajustes de Objects
        #df['dt_emissao'] = df['dt_emissao'].str.split('T').str[0]
        #df['dt_emissao'] = pd.to_datetime(df['dt_emissao'], format="%Y-%m-%d")
        #df[['vl_unit', 'sub_total']] = df[['vl_unit', 'sub_total']].replace({'\.':'', ',':''}, regex=True).astype(float) / 100
        #df['qtd'] = df['qtd'].str.split('\.').str[0].astype(int)
        
        return df


    def extracao_nf_versao_4_0():
        serie_nfe = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[4].text
        num_nfe = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[5].text
        dt_emissao = root.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[6].text
        tipo_comprador = root.getchildren()[0].getchildren()[0].getchildren()[2].getchildren()[0].tag
        tipo_comprador = tipo_comprador.replace('{http://www.portalfiscal.inf.br/nfe}', '')
        uf_comprador = root.getchildren()[0].getchildren()[0].getchildren()[2].getchildren()[2].findall('{http://www.portalfiscal.inf.br/nfe}UF')[0].text
        v_frete = float(root.getchildren()[0].getchildren()[0].findall('{http://www.portalfiscal.inf.br/nfe}total')[0].getchildren()[0].findall('{http://www.portalfiscal.inf.br/nfe}vFrete')[0].text)
        # Extração de Produtos do XML
        container =  root.getchildren()[0].getchildren()[0].getchildren()
        produtos = []
        for element in container:
            if element.tag == '{http://www.portalfiscal.inf.br/nfe}det':
                produtos.append(element)
        lst_sku_prod  = []
        lst_nome_prod = []
        lst_qtd_prod  = []
        lst_vl_unit_prod = []
        lst_sub_total_prod = []
        for prod in produtos:
            sku_prod = prod.getchildren()[0].getchildren()[0].text
            lst_sku_prod.append(sku_prod)
            nome_prod = prod.getchildren()[0].getchildren()[2].text
            lst_nome_prod.append(nome_prod)
            qtd_prod = prod.getchildren()[0].getchildren()[6].text
            lst_qtd_prod.append(qtd_prod)

            vl_unit_prod = prod.getchildren()[0].getchildren()[7].text
            lst_vl_unit_prod.append(vl_unit_prod)
            sub_total_prod = prod.getchildren()[0].getchildren()[8].text
            lst_sub_total_prod.append(sub_total_prod)

            # Montagem DF
        lista_de_colunas_df = ['xml_version', 'dt_emissao', 'num_nfe', 'tipo_comprador', 'uf', 'sku_prod', 'nome_prod', 'qtd', 'vl_unit', 'sub_total', ]
        df = pd.DataFrame(index=range(len(lst_sku_prod) ), columns=lista_de_colunas_df)
        df['xml_version'] = versao_xml
        df['dt_emissao'] = dt_emissao
        df['num_nfe'] = num_nfe
        df['tipo_comprador'] = tipo_comprador
        df['uf'] = uf_comprador
        df['sku_prod'] = lst_sku_prod
        df['nome_prod'] = lst_nome_prod
        df['qtd'] = lst_qtd_prod
        df['vl_unit'] = lst_vl_unit_prod
        df['sub_total'] = lst_sub_total_prod
        #Ajustes de Objects
        #df['dt_emissao'] = df['dt_emissao'].str.split('T').str[0]
        #df['dt_emissao'] = pd.to_datetime(df['dt_emissao'], format="%Y-%m-%d")
        #df[['vl_unit', 'sub_total']] = df[['vl_unit', 'sub_total']].replace({'\.':'', ',':''}, regex=True).astype(float) / 100
        #df['qtd'] = df['qtd'].str.split('\.').str[0].astype(int)
        
        return df

  
    arr = os.listdir('data')
    df = pd.DataFrame()

    # Progressbar
    progress_bar_init = 1
    progress_bar_final = len(arr)

    today = dt.today()
    file_name_pickle = f'DataFrame-Generated-{today}.pkl'
    
    with open('out/log.csv', 'w') as f: # Escreve arquivo com log do serviço executado, já que o print não funciona.
        for xml in arr:
            tree = et.parse(f"./data/{xml}")
            #print(f"{progress_bar_init}/{progress_bar_final}, {xml}")
            f.write(f"{progress_bar_init}/{progress_bar_final}, {xml}\n")
            
            root =  tree.getroot()
            versao_xml = root.getchildren()[0].getchildren()[0].get('versao')
            versao = versao_xml.split(r'.')[0]
            if versao == '2':
                a = extracao_nf_versao_2_0()
                df = pd.concat([df, a], ignore_index=True, axis=0)
            elif versao == '3':
                a = extracao_nf_versao_3_0()
                df = pd.concat([df, a], ignore_index=True, axis=0)
            elif versao == '1':
                a = extracao_nf_versao_1_0()
                df = pd.concat([df, a], ignore_index=True, axis=0)
            else:
                a = extracao_nf_versao_4_0()
                df = pd.concat([df, a], ignore_index=True, axis=0)
            progress_bar_init += 1


    df.to_pickle(f'out/{file_name_pickle}')
    return df

if __name__ == '__main__':
    parse_all_xml()

In [7]:
# Leitura do Ultimo arquivo de log
import pandas as pd
pd.options.display.max_colwidth = 100 #largura de Cada Colun
df = pd.read_csv('out/log.csv', names=['Progress', 'Xml'])
df.tail(1)['Xml']
last_file = df.iloc[-1]['Xml'].strip()
last_file
df.tail(1)['Xml']

115770     35220460955325000150550010000980691394028889-nfe.xml
Name: Xml, dtype: object

In [5]:
pd.read_pickle('out/DataFrame-Generated-2022-05-12.pkl').shape

(274149, 10)